In [1]:
import os
import difflib
import shutil
from glob import glob
import pandas as pd
import numpy as np
import operator

import music21

from utils import get_chord_progressions, calculate_melody_results, flatten_chord_progression
from melody import Melody

source = 'Jazz-Midi'

input_folder = f'../data/Complete Examples Melodies Auto/v1.2/{source}'
filepaths = [y for x in os.walk(input_folder) for y in glob(os.path.join(x[0], '*.mid'))]
cps = get_chord_progressions('..')

In [2]:
file_idx = 32

In [5]:
filepaths[file_idx:40]

['../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Yours Is My Heart Alone.mid']

In [4]:
filename = filepaths[file_idx]

all_hc = {}
for t in range(-6,6):
    melody = Melody(filename, '1.2', '..')
    melody.setup()
    melody.set_song_structure(cps[melody.song_name])
    melody.align_key()
    
#     melody.tempo /= 2
    
    melody.transpose_semitones = t
    
    melody.parse_notes()

    # print(melody.note_info)

    if len(melody.errors) > 0:
        raise Exception(melody.errors[0])

    for starting_measure in range(0, melody.n_chord_prog_measures):
        melody.starting_measure = starting_measure
        melody.split_note_info = []

        melody.split_melody(quantized=True, save=False)

        results = calculate_melody_results(melody)
        
#         for s in melody.split_note_info:
#             s.loc[:, 'end_ticks'] = s['ticks'] + s['duration']
#             melody.remove_strong_polyphony(s)
#             melody.remove_weak_polyphony(s)

        hc = np.array([])
        for k, res in results.items():
            hc = np.append(hc, res['harmonic_consistency_mean']) 

        all_hc[str(t) + '_' + str(starting_measure)] = (hc.mean(), hc.max(), hc.min(), len(hc))
    
all_hc = sorted(all_hc.items(), key=operator.itemgetter(1), reverse=True)

for i in all_hc[:40]:
    print(i)
print('----------------')

KeyError: 'Yours Is My Heart Alone'

In [5]:
# tra_semi = all_hc[correct_index][0].split('_')[0]
# starting = all_hc[correct_index][0].split('_')[1]

correct_index = 0
tra_semi = 5
starting = 6

melody = Melody(filename, '1.2', '..')
melody.setup()
melody.set_song_structure(cps[melody.song_name])
melody.align_key()

# melody.tempo /= 2

melody.transpose_semitones = int(tra_semi)
melody.starting_measure = int(starting)

melody.parse_notes()

melody.split_melody(quantized=True, save=False)

repetitions = len(melody.split_note_info)
# repetitions = 

print(melody.song_name)
print(f'Transpose semitones: {melody.transpose_semitones}')
print(f'Starting measure: {melody.starting_measure}')

print(f'N. repetitions: {repetitions}')

for r in range(1, repetitions + 1):
    melody.save_split_melody(repetition=r, quantized=True, chords=True)

You Are Too Beautiful
Transpose semitones: 5
Starting measure: 6
N. repetitions: 1


In [6]:
all_results = {}
results = calculate_melody_results(melody)

for k, v in results.items():
    all_results_key = melody.filename + ' -' + str(k) + '-'
    all_results[all_results_key] = v

In [7]:
gtg = input('Good to go? ')

if gtg == 'g':
    
    print('Finalising')
    
    for k, results in all_results.items():
        folder_midi = f'../data/split_melody/v1.2/{source}/'
        folder_csv = f'../data/split_melody_data/v1.2/{source}/'

        src_midi = folder_midi + results['out_filename']
        dst_midi = src_midi.replace('split_melody/v1.2', 'finalised/midi')

        src_csv = folder_csv + results['out_filename'].replace('.mid', '.csv')
        dst_csv = src_csv.replace('split_melody_data/v1.2', 'finalised/csv')

        if not os.path.exists(os.path.dirname(dst_midi)):
            os.makedirs(os.path.dirname(dst_midi))

        if not os.path.exists(os.path.dirname(dst_csv)):
            os.makedirs(os.path.dirname(dst_csv))

        shutil.copyfile(src_midi, dst_midi)
        shutil.copyfile(src_csv, dst_csv)

Good to go? g
Finalising
